**Have used the sklearn library only for the iris dataset, no further use of it will be done.**

In [99]:
from sklearn import datasets
iris=datasets.load_iris()
X=iris.data
y=iris.target

The only library we will be using for the entirety of the task.

In [100]:
import time

The randon_gen() function will generate a random number (integer), which will either be added or subtracted based on a condition.

In [101]:
def random_gen():
  current=time.time()
  last_digit=int(str(current-int(current))[-1])
  if last_digit%2==0:
    return last_digit
  else:
    return -1*last_digit

We need to convert the data and the targets to lists since they are originally ndarrays, and we will not be using the numpy library anywhere

In [102]:
#Convert both variables to list since originally they are ndarrays
X=X.tolist()
y=y.tolist()

In [103]:
#Helper Functions defined here:

#Function for natural log, later can be used for log base 10. Can also try the Taylor Definition, but this seemed easier. I will compare precision later.
def ln(x):
    n = 10000000.0
    return n * ((x ** (1/n)) - 1)

#Can convert to base 10
def log(x):
  return ln(x)/ln(10)

#Function to calculate derivative of a function
def derivative(value,function):
  step=0.000001
  return (function(value+step) - function(value-step))/(2*step)

#Function to calculate the sigmoid
e=2.71828
def sigmoid(value):
  return 1/(1+e**(-1*value))

#Function to calculate the derivative of Binary Cross Entropy Loss
def bce_loss_derivative(y_true,z):
  return (y_true-z)/(z)*(1-z)

#Function to calculate the derivative of the sigmoid function
def sigmoid_derivative(value):
  return sigmoid(value) * (1-sigmoid(value))

#Function to calculate the Binary Cross Entropy Loss
def bce_loss(value,y_true):
  return -1*((y_true * log(value)) + ((1-y_true) * log(1-value)))

#Function to calculate the softmax
def softmax(layer):
  sum=0
  for i in range(len(layer)):
    sum+= 2.71828**layer[i]
  for i in range(len(layer)):
    layer[i]=2.71828**layer[i]/sum

#Function to calculate categorical loss
def categorical_loss(layer,y_true):
  loss=log(layer[y_true])
  return loss

#Function to calculate derivative of categorical loss
def cl_derivative(value,y_true):
  one_hot=[0,0,0]
  one_hot[y_true]=1
  return value-one_hot[0]

#Function to take deepcopy of an object
def deep_copy(obj):
    if isinstance(obj, list):
        return [deep_copy(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: deep_copy(value) for key, value in obj.items()}
    else:
        return obj

This is just to see what kind of classification problem is there, and the number of training examples, and features.

In [104]:
print("The number of training examples are : " + str(len(X)))
print("The number of attributes/features are : " + str(len(X[0])))
print("The number of unique classes are : " + str(len(set(y))))

The number of training examples are : 150
The number of attributes/features are : 4
The number of unique classes are : 3


Since there are 3 classes, this becomes a multi-class classification problem.
Hence, we can structure our architecture in such a way that the final layer has 3 neurons, over which we can apply softmax, and then further find the loss using categorical loss.

In [105]:
X

[[5.1, 3.5, 1.4, 0.2],
 [4.9, 3.0, 1.4, 0.2],
 [4.7, 3.2, 1.3, 0.2],
 [4.6, 3.1, 1.5, 0.2],
 [5.0, 3.6, 1.4, 0.2],
 [5.4, 3.9, 1.7, 0.4],
 [4.6, 3.4, 1.4, 0.3],
 [5.0, 3.4, 1.5, 0.2],
 [4.4, 2.9, 1.4, 0.2],
 [4.9, 3.1, 1.5, 0.1],
 [5.4, 3.7, 1.5, 0.2],
 [4.8, 3.4, 1.6, 0.2],
 [4.8, 3.0, 1.4, 0.1],
 [4.3, 3.0, 1.1, 0.1],
 [5.8, 4.0, 1.2, 0.2],
 [5.7, 4.4, 1.5, 0.4],
 [5.4, 3.9, 1.3, 0.4],
 [5.1, 3.5, 1.4, 0.3],
 [5.7, 3.8, 1.7, 0.3],
 [5.1, 3.8, 1.5, 0.3],
 [5.4, 3.4, 1.7, 0.2],
 [5.1, 3.7, 1.5, 0.4],
 [4.6, 3.6, 1.0, 0.2],
 [5.1, 3.3, 1.7, 0.5],
 [4.8, 3.4, 1.9, 0.2],
 [5.0, 3.0, 1.6, 0.2],
 [5.0, 3.4, 1.6, 0.4],
 [5.2, 3.5, 1.5, 0.2],
 [5.2, 3.4, 1.4, 0.2],
 [4.7, 3.2, 1.6, 0.2],
 [4.8, 3.1, 1.6, 0.2],
 [5.4, 3.4, 1.5, 0.4],
 [5.2, 4.1, 1.5, 0.1],
 [5.5, 4.2, 1.4, 0.2],
 [4.9, 3.1, 1.5, 0.2],
 [5.0, 3.2, 1.2, 0.2],
 [5.5, 3.5, 1.3, 0.2],
 [4.9, 3.6, 1.4, 0.1],
 [4.4, 3.0, 1.3, 0.2],
 [5.1, 3.4, 1.5, 0.2],
 [5.0, 3.5, 1.3, 0.3],
 [4.5, 2.3, 1.3, 0.3],
 [4.4, 3.2, 1.3, 0.2],
 [5.0, 3.5,

We now define the architecture. Feel free to play around with the values of num_layers, and add your own neurons per layer.

In [207]:
#Choose architecture here:
num_layers = 5
#That implies 1 input layer, 4 hidden layers, and 1 output layer

#######################################################
#no_of_neurons=[]
#for i in num_layers:
# l=int(input("Enter number of neurons :"))
# no_of_neurons.append(l)
#######################################################

no_of_neurons=[4,5,4,3,4]
final_layer=(len(set(y)))
no_of_neurons.append(final_layer)
n=no_of_neurons

In [107]:
n

[4, 5, 4, 3, 4, 3]

This is our architecture. The last layer as mentioned is the number of classes to help us find the softmax.

In [185]:
#define the weights here:
Weights=[]
for i in range(len(n)-1):
  current=time.time()
  m=int(str(current-int(current))[-1])
  Weights.append([[(m/10 + random_gen())/10 for x in range(n[i+1])] for y in range(n[i])])

In [186]:
#define the biases here:
Biases=[]
for i in range(len(n)-1):
  current=time.time()
  m=int(str(current-int(current))[-1])
  Biases.append([(m/10+ random_gen())/10 for x in range(n[i+1])])

In [187]:
#define the gradients of weights here:
Weights_Grad=[]
for i in range(len(n)-1):
  Weights_Grad.append([[0 for x in range(n[i+1])] for y in range(n[i])])

In [188]:
#define the gradients of biases here:
Biases_Grad=[]
for i in range(len(n)-1):
  Biases_Grad.append([0 for x in range(n[i+1])])

In [189]:
#define the activations here
Activations=[]
for i in range(len(n)-1):
  Activations.append([0 for x in range(n[i+1])])

In [190]:
#define cached activations here
ad=[]
for i in range(len(n)-1):
  ad.append([0 for x in range(n[i+1])])

Check the weights, biases here.
The gradients of weights as well as the activations are initialized to 0.

In [191]:
Weights

[[[-0.48,
   0.22000000000000003,
   -0.8800000000000001,
   -0.27999999999999997,
   -0.48],
  [-0.6799999999999999, -0.48, -0.27999999999999997, -0.48, 0.82],
  [0.62,
   -0.48,
   -0.6799999999999999,
   -0.8800000000000001,
   -0.27999999999999997],
  [-0.8800000000000001, -0.08, 0.42000000000000004, 0.82, -0.08]],
 [[-0.45999999999999996, 0.64, 0.8400000000000001, 0.24],
  [0.64, -0.86, -0.06, 0.44000000000000006],
  [0.44000000000000006, -0.06, -0.26, 0.64],
  [-0.06, -0.26, 0.64, -0.86],
  [-0.45999999999999996,
   -0.6599999999999999,
   -0.6599999999999999,
   -0.6599999999999999]],
 [[0.44000000000000006, 0.64, 0.64],
  [-0.26, -0.45999999999999996, -0.6599999999999999],
  [0.44000000000000006, 0.64, -0.26],
  [-0.45999999999999996, 0.64, -0.6599999999999999]],
 [[-0.82, 0.27999999999999997, -0.22000000000000003, 0.6799999999999999],
  [0.6799999999999999,
   -0.019999999999999997,
   0.27999999999999997,
   0.27999999999999997],
  [0.6799999999999999, -0.62, 0.88000000000000

In [192]:
Biases

[[-0.6900000000000001,
  0.21000000000000002,
  0.41,
  -0.49000000000000005,
  0.8099999999999999],
 [-0.05, -0.45, 0.65, -0.45],
 [-0.49000000000000005, -0.6900000000000001, -0.89],
 [-0.48, -0.48, 0.62, 0.62],
 [0.27, 0.27, -0.22999999999999998]]

In [193]:
Weights_Grad

[[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],
 [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],
 [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]]

In [194]:
Biases_Grad

[[0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0]]

In [195]:
Activations

[[0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0]]

In [196]:
w=Weights
b=Biases
w_grad=Weights_Grad
b_grad=Biases_Grad
a=Activations
#ca=Cached_Activations
w_original=deep_copy(w)
#b_original=deep_copy(b)

Split the dataset into train and test, into a 7:3 ratio. Hence 105 for training and 45 for testing.

Also check the number of instances of each class, to see if any methods need to be applied to mitigate the effects of class imbalance.

Here, advantage is taken of the fact that all classes occur in equal number, and hence slicing values are chosen in such a way to ensure we have an equal number of instances of each class in the training and testing data.

In [197]:
indices = [slice(0, 35), slice(50, 85), slice(100, 135)]
test_indices=[slice(35,50), slice(85,100), slice(135,150)]
X_train = []
y_train=[]
X_test=[]
y_test=[]

for id in indices:
    X_train += X[id]
    y_train += y[id]
for id in test_indices:
  X_test+= X[id]
  y_test+= y[id]

In [198]:
len(X_train), len(X_test)
#Checks out!

(105, 45)

In [199]:
count0=y.count(0)
count1=y.count(1)
count2=y.count(2)

In [200]:
count0,count1,count2
#(0-49) , (50,99) , (100,149)

(50, 50, 50)

In [176]:
ca=[]

Functions forward_pass and get_loss are defined here. Next is the function calculate_ad, which caches the values of dL/dZi.

In [201]:
def forward_pass(message,true_label,X):
  input_layer=X[message]
  for i in range(len(n)-2):
    for j in range(n[i+1]):
      for k in range(n[i]):
        a[i][j]+=w[i][k][j] * input_layer[k]
      a[i][j] += b[i][j]
      a[i][j] = sigmoid(a[i][j])
    input_layer = a[i]
  softmax_layer=a[-2]
  i=len(n)-2
  for j in range(n[i+1]):
    for k in range(n[i]):
      a[i][j]+=w[i][k][j] * input_layer[k]
    a[i][j]+=b[i][j]
  output_layer=a[-1]
  softmax(output_layer)
  t=deep_copy(a)
  loss=categorical_loss(output_layer,true_label)
  for xx in range(len(a)):
    for yy in range(len(a[xx])):
      a[xx][yy]=0
  return (loss,t)

def get_loss(X_train,y_train):
  net_loss=0
  ca=[]
  for message in range(len(X_train)):
    loss,t=forward_pass(message,y_train[message],X_train)
    net_loss+=loss
    ca.append(t)

  return (net_loss,ca)

In [202]:
def calculate_ad(layer):
  for i in range(len(w[layer])):
    z=0
    for message in range(len(X_train)):
      z=ad[layer][0]*sigmoid_derivative(cache[message][layer][0])*w[layer][i][0]
    z=z/len(X_train)
    ad[layer-1][i]=z

Run this to get loss per time step.

In [203]:
#l,cache=get_loss(X_train,y_train)

Defining the hyperparamter learning rate as 0.01 here.

In [204]:
learning_rate=0.01

Backpropogation starts here!

In [205]:
def backpropogation(cache):
  #Lets just update the last layer first
  for number_cache in range(len(cache[0][-1])):
    for number in range(len(w_grad[-1])):
      grad_w=0
      for message in range(len(X_train)):
        grad_w+=(cl_derivative(cache[message][-1][number_cache], y_train[message]) * cache[message][-2][number])
      grad_w=grad_w/len(X_train)
      w_grad[-1][number][number_cache]=grad_w
      w[-1][number][number_cache]-=(learning_rate*grad_w)
  for number_cache in range(len(cache[0][-1])):
    grad_b=0
    for message in range(len(X_train)):
      grad_b+=(cl_derivative(cache[message][-1][number_cache], y_train[message]))
    grad_b=grad_b/len(X_train)
    b_grad[-1][number_cache]=grad_b
    b[-1][number_cache]-=(learning_rate*grad_b)
  for f in range(len(w[-1])):
    z=0
    for message in range(len(X_train)):
      z+=(cl_derivative(cache[message][-1][0],y_train[message])) *w[-1][f][0]
    z=z/len(X_train)
    ad[-2][f]=z
  layer=-2
  while(layer>=-1*(len(n)-1)):
    for number_cache in range(len(cache[0][layer])):
      for number in range(len(w_grad[layer])):
        grad_w=0
        for message in range(len(X_train)):
          if layer==-1*(len(n)-1):
            grad_w+=ad[layer][number_cache] * sigmoid_derivative(cache[message][layer][number_cache])*X_train[message][number]
          else:
            grad_w+=ad[layer][number_cache] * sigmoid_derivative(cache[message][layer][number_cache])*cache[message][layer-1][number]
        grad_w=grad_w/len(X_train)
        w_grad[layer][number][number_cache]=grad_w
        w[layer][number][number_cache]-=(learning_rate*grad_w)
    if layer==-1*(len(n)-1):
      break
    else:
      calculate_ad(layer)
    layer=layer-1
  layer_b=-2
  while(layer_b>=-1*(len(n)-1)):
    for number_cache in range(len(cache[0][layer_b])):
      grad_b2=0
      for message in range(len(X_train)):
        grad_b2+=ad[layer][number_cache] * sigmoid_derivative(cache[message][layer][number_cache])
      grad_b2=grad_b2/len(X_train)
      b_grad[layer_b][number_cache]=grad_b2
      b[layer_b][number_cache]-=(learning_rate*grad_b2)
    layer_b=layer_b-1

-------------------------------------------------------------------------------------------------

Manually carrying out the backpropogation for the last layer, to check and verify.

In [156]:
#Lets just update the last layer first
for number_cache in range(len(cache[0][-1])):
  for number in range(len(w_grad[-1])):
    grad_w=0
    for message in range(len(X_train)):
      grad_w+=(cl_derivative(cache[message][-1][number_cache], y_train[message]) * cache[message][-2][number])
    grad_w=grad_w/len(X_train)
    w_grad[-1][number][number_cache]=grad_w
    w[-1][number][number_cache]-=(learning_rate*grad_w)

In [157]:
for number_cache in range(len(cache[0][-1])):
  grad_b=0
  for message in range(len(X_train)):
    grad_b+=(cl_derivative(cache[message][-1][number_cache], y_train[message]))
  grad_b=grad_b/len(X_train)
  b_grad[-1][number_cache]=grad_b
  b[-1][number_cache]-=(learning_rate*grad_b)

In [158]:
for f in range(len(w[-1])):
  z=0
  for message in range(len(X_train)):
    z+=(cl_derivative(cache[message][-1][0],y_train[message])) *w[-1][f][0]
  z=z/len(X_train)
  ad[-2][f]=z

In [133]:
#Check the last cell for the updates in the weights and biases
print(w[-1])
print(b[-1])

[[0.2692583617286699, 0.2703632047017512, -0.029610950056028186], [-0.4306192137088341, 0.47029370477289995, -0.22968493993792488], [-0.4306182266023227, -0.2297037971152416, -0.22968240637974965], [-0.8302586534333463, -0.6298773626084666, 0.6701315566695254]]
[0.6485406225097313, -0.4492956191371982, -0.24924500337253314]


-----------------------------------------------------------------------

Now, both forward and backward passes are defined, I will run this for 1000 timesteps. You can also track w_grad and b_grad if you wish to, can add print in any fucntion to just display the two lists

In [206]:
for i in range(1000):
  l,cache=get_loss(X_train,y_train)
  print(str(i)+" : " + str(l))
  backpropogation(cache)

0 : -55.91349644514608
1 : -55.843349362003316
2 : -55.7739311396555
3 : -55.70523557120951
4 : -55.63725651100714
5 : -55.56998786691037
6 : -55.50342359740822
7 : -55.43755771788522
8 : -55.37238429049561
9 : -55.307897430431744
10 : -55.24409130737036
11 : -55.180960131972284
12 : -55.118498163114545
13 : -55.05669970403328
14 : -54.9955591076275
15 : -54.93507076970881
16 : -54.87522912466185
17 : -54.81602865605196
18 : -54.75746387878522
19 : -54.69952935598393
20 : -54.642219689343555
21 : -54.585529516721884
22 : -54.52945351262102
23 : -54.473986393973604
24 : -54.419122912428016
25 : -54.36485785290184
26 : -54.31118603165346
27 : -54.2581023155683
28 : -54.20560158985409
29 : -54.153678781666834
30 : -54.10232884130623
31 : -54.05154676728848
32 : -54.0013275778985
33 : -53.951666321315344
34 : -53.902558090076994
35 : -53.85399799745466
36 : -53.80598118709578
37 : -53.75850283447051
38 : -53.71155814060392
39 : -53.66514235570145
40 : -53.61925072900447
41 : -53.5738785550

The loss can be seen to decrease with timesteps. Note that the architecture is very small, with very few neruons, and optimization techniques are not employed, hence there will be a trail in the decrease of loss.

-----------------------------------------------------------------------------------------------